In [1]:
import pandas as pd

In [2]:
# now that we have the df for twitter sentiment of each day fron 20170101
sentiment_filename = 'dailysentiments.csv'
stock_filename = 'stockprice_raw.csv'
sentiments = pd.read_csv(sentiment_filename, engine='python',index_col='date')
stocks = pd.read_csv(stock_filename, engine = 'python',index_col='Date')
sentiments = sentiments.drop(columns=sentiments.columns[0])

In [3]:
# set the size of the sliding window of stock price prediction
window_size = 1
# use Open_prev to find the opening price of the previous day
stocks['Open_prev'] = stocks['Open'].shift(window_size)

In [4]:
from datetime import timedelta
dates = stocks.index
dates = pd.to_datetime(dates)

#to correct for if the gap is more than one day
for k in range(stocks.shape[0]):
  if k > 0 and (dates[k] - dates[k-1]) > timedelta(days =1):
    idx = stocks.index[k]
    stocks.at[idx, 'Open_prev'] = stocks.iloc[k-1]['Open']

In [5]:
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_prev
Date,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,43.259998
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.972000
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,42.950001
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.284000
...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,642.989990
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,674.510010
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,661.000000


In [6]:
stocks['Diff'] = stocks['Close'] - stocks['Open_prev']
stocks.loc[stocks['Diff']> 0, 'Diff'] = 1
stocks.loc[stocks['Diff']< 0, 'Diff'] = -1
stocks.loc[stocks['Diff'] == 0, 'Diff'] = 0
stock_label_df = stocks[['date','Diff']]

In [7]:
stock_label_df
#stock_label_df.to_csv('stockprice_label.csv')

,date,Diff
Date,,
2016-12-30,20161230,NaN
2017-01-03,20170103,1.0
2017-01-04,20170104,1.0
2017-01-05,20170105,1.0
2017-01-06,20170106,1.0
...,...,...
2020-12-28,20201228,1.0
2020-12-29,20201229,-1.0
2020-12-30,20201230,1.0


In [8]:
'''
previous code:
# Create a df with sentiment analysis from previous two days given the dateindex 
sentiment_prev2days = pd.concat([sentiments.shift(1),sentiments.shift(2)],axis=1)
sentiment_prev2days = sentiment_prev2days.dropna()
sentiment_prev2days.insert(0,'date', sentiment_prev2days.index)
sentiment_prev2days.index.names = ['Date']
sentiment_prev2days
# note that 1448 tweet days vs. 1007 valid stock labels

'''

#sentiment analysis window
sentiment_window_size = 7

#first shift to the day before stock price window
sentiment_shift = sentiments.shift(window_size+1)

#accumulate features of sentiment analysis window
for i in range(3, sentiment_window_size + 2):
  sentiment_shift = pd.concat([sentiment_shift,sentiments.shift(i)],axis=1)

sentiment_shift = sentiment_shift.dropna()

sentiment_shift

,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet,pos_mean_replies,pos_mean_retweet,...,neg_mean_replies,neg_mean_retweet,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet
date,,,,,,,,,,,,,,,,,,,,,
20170109,0.552184,0.598106,2.431541,2.477603,2.434837,0.209339,0.196020,0.192863,2.165374,2.700563,...,0.858484,1.255735,3.828412,3.232838,8.028677,8.366872,9.094131,0.264051,0.304716,0.173031
20170110,2.973540,3.970256,4.156664,5.277351,4.300867,0.074348,0.082443,0.062210,0.552184,0.598106,...,4.482420,4.210966,6.260311,5.908047,6.780715,7.166920,7.121932,1.182442,0.858484,1.255735
20170111,1.733815,4.389617,6.720076,3.673627,5.217909,4.781877,8.878273,4.678189,2.973540,3.970256,...,2.996988,0.365056,1.469026,1.436088,4.423724,4.048554,4.352946,4.342607,4.482420,4.210966
20170112,2.980973,3.052879,2.571007,2.997780,3.021332,0.138117,0.271247,0.175789,1.733815,4.389617,...,0.598628,0.552180,0.382300,0.921218,4.941648,2.870712,4.963726,0.486562,2.996988,0.365056
20170113,2.520085,2.844737,5.373914,5.619566,5.162004,0.271010,0.193683,0.326593,2.980973,3.052879,...,0.207291,0.216844,2.516691,2.811572,6.303775,5.975591,5.727157,0.566462,0.598628,0.552180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,0.461738,0.186157,0.448576,0.485939,0.455544,0.318151,0.052323,0.358299,0.415584,0.355880,...,0.182667,0.091325,0.297707,0.382962,0.397160,0.551848,0.416572,0.285088,0.150445,0.200466
20201228,0.409077,0.364218,0.397077,0.424900,0.539451,0.286412,0.166023,0.096330,0.461738,0.186157,...,0.104569,0.085256,0.267194,0.323698,0.482170,0.550139,0.584976,0.114582,0.182667,0.091325
20201229,0.862483,0.781388,0.093095,0.110687,0.179754,0.020394,0.026830,0.038858,0.409077,0.364218,...,0.501292,0.508401,0.383977,0.334963,0.367910,0.351789,0.420118,0.073144,0.104569,0.085256


In [9]:
dataframe = sentiment_shift.merge(stock_label_df,how='left', left_on='date', right_on='date')
dataframe = dataframe.dropna()
dataframe

,date,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet,pos_mean_replies,...,neg_mean_retweet,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet,Diff
0,20170109,0.552184,0.598106,2.431541,2.477603,2.434837,0.209339,0.196020,0.192863,2.165374,...,1.255735,3.828412,3.232838,8.028677,8.366872,9.094131,0.264051,0.304716,0.173031,1.0
1,20170110,2.973540,3.970256,4.156664,5.277351,4.300867,0.074348,0.082443,0.062210,0.552184,...,4.210966,6.260311,5.908047,6.780715,7.166920,7.121932,1.182442,0.858484,1.255735,1.0
2,20170111,1.733815,4.389617,6.720076,3.673627,5.217909,4.781877,8.878273,4.678189,2.973540,...,0.365056,1.469026,1.436088,4.423724,4.048554,4.352946,4.342607,4.482420,4.210966,-1.0
3,20170112,2.980973,3.052879,2.571007,2.997780,3.021332,0.138117,0.271247,0.175789,1.733815,...,0.552180,0.382300,0.921218,4.941648,2.870712,4.963726,0.486562,2.996988,0.365056,1.0
4,20170113,2.520085,2.844737,5.373914,5.619566,5.162004,0.271010,0.193683,0.326593,2.980973,...,0.216844,2.516691,2.811572,6.303775,5.975591,5.727157,0.566462,0.598628,0.552180,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,20201224,0.075541,0.060729,0.404326,0.423167,0.430870,0.523092,0.501292,0.508401,0.383977,...,0.098894,0.232098,0.134903,0.634783,0.643418,0.779769,0.117921,0.124484,0.085328,1.0
1423,20201228,0.409077,0.364218,0.397077,0.424900,0.539451,0.286412,0.166023,0.096330,0.461738,...,0.085256,0.267194,0.323698,0.482170,0.550139,0.584976,0.114582,0.182667,0.091325,1.0
1424,20201229,0.862483,0.781388,0.093095,0.110687,0.179754,0.020394,0.026830,0.038858,0.409077,...,0.508401,0.383977,0.334963,0.367910,0.351789,0.420118,0.073144,0.104569,0.085256,-1.0
1425,20201230,0.117387,0.041629,0.310698,0.787195,0.255934,0.634153,0.095418,0.702437,0.862483,...,0.483703,0.075541,0.060729,0.404326,0.423167,0.430870,0.523092,0.501292,0.508401,1.0


In [10]:

filename = "database_prev{}days.csv".format(sentiment_window_size)
filename

'database_prev7days.csv'

In [11]:
dataframe.to_csv(filename)